In [2]:
import cv2 
import numpy as np
import sys
import os
import argparse
import albumentations as A
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from PIL import Image



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-p

In [1]:
# find number of images in a directory
def count_images_in_directory(directory):
    count = 0
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            count += 1
    return count

# find leaf directories in a given directory
def find_leaf_directories(directory):
    leaf_dirs = []
    for root, dirs, files in os.walk(directory):
        if not dirs:  # if there are no subdirectories
            leaf_dirs.append(root)
    return leaf_dirs

In [4]:
leaf_dirs_images_images= find_leaf_directories('Dataset/images/train')
leaf_dirs_images_images

['Dataset/images/train/sad',
 'Dataset/images/train/fear',
 'Dataset/images/train/surprise',
 'Dataset/images/train/neutral',
 'Dataset/images/train/disgust',
 'Dataset/images/train/happy',
 'Dataset/images/train/angry']

In [5]:
for leaf_dir in leaf_dirs_images_images:
    image_count = count_images_in_directory(leaf_dir)
    print(f"Directory: {leaf_dir}, Number of images: {image_count}")

Directory: Dataset/images/train/sad, Number of images: 4938
Directory: Dataset/images/train/fear, Number of images: 4103
Directory: Dataset/images/train/surprise, Number of images: 3205
Directory: Dataset/images/train/neutral, Number of images: 4982
Directory: Dataset/images/train/disgust, Number of images: 436
Directory: Dataset/images/train/happy, Number of images: 7164
Directory: Dataset/images/train/angry, Number of images: 3993


In [6]:
leaf_dirs_images_train = find_leaf_directories('Dataset/images/validation')
leaf_dirs_images_train

['Dataset/images/validation/sad',
 'Dataset/images/validation/fear',
 'Dataset/images/validation/surprise',
 'Dataset/images/validation/neutral',
 'Dataset/images/validation/disgust',
 'Dataset/images/validation/happy',
 'Dataset/images/validation/angry']

In [7]:
for leaf_dir in leaf_dirs_images_train:
    image_count = count_images_in_directory(leaf_dir)
    print(f"Directory: {leaf_dir}, Number of images: {image_count}")

Directory: Dataset/images/validation/sad, Number of images: 1139
Directory: Dataset/images/validation/fear, Number of images: 1018
Directory: Dataset/images/validation/surprise, Number of images: 797
Directory: Dataset/images/validation/neutral, Number of images: 1216
Directory: Dataset/images/validation/disgust, Number of images: 111
Directory: Dataset/images/validation/happy, Number of images: 1825
Directory: Dataset/images/validation/angry, Number of images: 960


In [8]:
sorted(os.listdir('Dataset/images/train'))


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [9]:
# find out the pixel size of the images in the directory
def find_image_size(directory):
    sizes = set()
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
            sizes.add(img.size)
    return sizes

image_sizes = find_image_size('Dataset/images/train/fear')
print(f"Image sizes in the directory: {image_sizes}")

Image sizes in the directory: {(48, 48)}


In [10]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
from deepface import DeepFace
from torch.utils.data import Dataset




/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-05 12:15:55.356901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754376355.370895   11511 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754376355.375192   11511 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754376355.387420   11511 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the s

In [11]:
class EmotionDataset(Dataset):
    def __init__(self, root_dir,embedder, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []
        for idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith(('png', 'jpg', 'jpeg')):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(idx)
       
        self.embedder = embedder  # Keras model

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):

        # Add this in a TRy Except block to handle errors

        try:
            img_path = self.image_paths[idx]
            label = self.labels[idx]
            print(f"Processing image {img_path} with label {label}")
            # Load grayscale then convert to RGB
            img = Image.open(img_path).convert('L').convert('RGB')

            # covert this to a numpy array
            img = np.array(img)
            
            representations = DeepFace.represent(
                img_path=img,
                model_name='VGG-Face',
                detector_backend='mtcnn'  # A robust face detector
                )

            # The result is a list of dictionaries, one for each face detected in the image.
            # We'll assume there is only one face for this example.
            if representations:
                embedding = representations[0]['embedding']
                
            # Convert to torch tensor
            embedding = torch.tensor(embedding, dtype=torch.float)
            label = torch.tensor(label, dtype=torch.long)
        
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            embedding = torch.zeros(4096, dtype=torch.float)  # Assuming 2622 is the embedding size
            label = -1  # Use -1 or some other value to indicate an error
        
        print(f"Processed image {img_path} with label {label}")
        return embedding, label
       


In [2]:
import torch.nn as nn

class EmotionClassifier(nn.Module):
    def __init__(self, embedding_size, num_classes):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(embedding_size, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/media/sudip/linux-extra/code/almabetter/deepFER/.venv/lib/python3.11/site-p

In [ ]:
def train_model(data_dir,
                num_epochs=3,
                batch_size=500,
                lr=1e-3,
                save_path='emotion_detector_deepface.pth'):
    # MTCNN for face detection
    # mtcnn = MTCNN(image_size=160, margin=0)
    # Load DeepFace embedding model
    print("Loading DeepFace embedding model...")

    embedder = DeepFace.build_model('VGG-Face') 
    # Dataset + DataLoader
    dataset = EmotionDataset(root_dir=data_dir,
                            #  mtcnn=mtcnn,
                             embedder=embedder)
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=5)
    
    print(f"Dataset loaded with {len(dataset)} samples.")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = EmotionClassifier(embedding_size=4096,  # VGG-Face embedding size
                              num_classes=len(dataset.classes))
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for embeddings, labels in dataloader:
            embeddings = embeddings.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * embeddings.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}')

    # Save classifier state and classes
    torch.save({
        'model_state_dict': model.state_dict(),
        'class_names': dataset.classes
    }, save_path)
    print(f'Model saved to {save_path}')




In [14]:
   
data_dir = 'Dataset/images/train'
num_epochs = 20
batch_size = 32
lr = 1e-3   
save_path = 'emotion_detector_deepface.pth'


train_model(
        data_dir=data_dir,
        num_epochs=num_epochs,
        batch_size=batch_size,
        lr=lr,
        save_path=save_path
    )

W0000 00:00:1754376358.798349   11511 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


KeyboardInterrupt: 

In [ ]:
dataset = EmotionDataset(
    root_dir='Dataset/images/train',
    embedder=DeepFace.build_model('VGG-Face')
)

# find the set of labels in the dataset
labels = set(dataset.labels)
print(f"Unique labels in the dataset: {labels}")

Unique labels in the dataset: {0, 1, 2, 3, 4, 5, 6}


In [22]:
import numpy as np
from deepface import DeepFace
import cv2

# Load an image using OpenCV
# You can replace 'path/to/your/image.jpg' with the actual path to your image
# img = cv2.imread('Dataset/images/train/fear/35720.jpg')  # Example image path
img = cv2.imread('Dataset/images/train/neutral/32462.jpg') 

# Check if the image was loaded successfully
if img is None:
    print("Error: Image not found or could not be loaded.")
else:
    # Get the embedding using DeepFace.represent()
    # The 'model_name' parameter specifies which model to use.
    # 'VGG-Face' is the default and a good choice.
    # The 'detector_backend' parameter can be changed if you want a different face detector.
    # 'opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe' are available.
    representations = DeepFace.represent(
        img_path=img,
        model_name='VGG-Face',
        detector_backend='mtcnn'  # A robust face detector
    )

    # The result is a list of dictionaries, one for each face detected in the image.
    # We'll assume there is only one face for this example.
    if representations:
        embedding = representations[0]['embedding']
        print("Embedding shape:", np.array(embedding).shape)
        print("Embedding (first 10 values):", embedding[:10])
        print("Image path:", img    )

        # The size of the VGG-Face embedding is 2622
        # This is a fixed-size vector representing the face.
        print("Embedding size:", len(embedding))
    else:
        print("No face detected in the image.")

Embedding shape: (4096,)
Embedding (first 10 values): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Image path: [[[254 254 254]
  [244 244 244]
  [174 174 174]
  ...
  [140 140 140]
  [235 235 235]
  [255 255 255]]

 [[254 254 254]
  [232 232 232]
  [136 136 136]
  ...
  [124 124 124]
  [185 185 185]
  [250 250 250]]

 [[254 254 254]
  [220 220 220]
  [100 100 100]
  ...
  [124 124 124]
  [136 136 136]
  [218 218 218]]

 ...

 [[ 43  43  43]
  [ 42  42  42]
  [ 42  42  42]
  ...
  [ 38  38  38]
  [ 47  47  47]
  [ 49  49  49]]

 [[ 42  42  42]
  [ 42  42  42]
  [ 38  38  38]
  ...
  [ 41  41  41]
  [ 43  43  43]
  [ 37  37  37]]

 [[ 37  37  37]
  [ 41  41  41]
  [ 36  36  36]
  ...
  [ 32  32  32]
  [ 37  37  37]
  [ 34  34  34]]]
Embedding size: 4096


In [5]:
import torch
import os

# set torch device to use gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load dee-face-emotion-detector-classifier model
model_path = 'emotion_detector_deepface.pth'
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model = EmotionClassifier(embedding_size=4096, num_classes=len(checkpoint['class_names']))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    class_names = checkpoint['class_names']
    print(f"Model loaded successfully with classes: {class_names}")

Model loaded successfully with classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [12]:
import cv2
import numpy as np
from deepface import DeepFace
# valdate the model on a sample image
img_path = 'Dataset/images/validation/fear/21.jpg'  # Example image path
img = cv2.imread(img_path)
if img is None:
    print("Error: Image not found or could not be loaded.")
else:   
    # Get the embedding using DeepFace.represent()
    representations = DeepFace.represent(
        img_path=img_path,
        model_name='VGG-Face',
        detector_backend='mtcnn'  # A robust face detector
    )

    if representations:
        print("device:", device)
        model.to(device)
        embedding = representations[0]['embedding']
        embedding_tensor = torch.tensor(embedding, dtype=torch.float).unsqueeze(0).to(device)

        # Forward pass through the model
        with torch.no_grad():
            output = model(embedding_tensor)
            _, predicted = torch.max(output, 1)
            predicted_class = class_names[predicted.item()]

        print(f"Predicted class for the image {img_path}: {predicted_class}")
    else:
        print("No face detected in the image.")

device: cuda
Predicted class for the image Dataset/images/validation/fear/21.jpg: fear


In [ ]:
# Function to predict emotion from an image

def predict_emotion(image_path, model):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found or could not be loaded.")
        return None

    representations = DeepFace.represent(
        img_path=image_path,
        model_name='VGG-Face',
        detector_backend='mtcnn'
    )

    if representations:
        embedding = representations[0]['embedding']
        embedding_tensor = torch.tensor(embedding, dtype=torch.float).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(embedding_tensor)
            _, predicted = torch.max(output, 1)
            predicted_class = class_names[predicted.item()]

        return predicted_class
    else:
        print("No face detected in the image.")
        return None
    



In [29]:
# call function to predict emotion from an image

image_path = ['Dataset/images/validation/fear/21.jpg',
              'Dataset/images/validation/happy/331.jpg',
              'Dataset/images/validation/sad/822.jpg',
              'Dataset/images/validation/neutral/1045.jpg',
              'Dataset/images/validation/surprise/3874.jpg',
              'Dataset/images/validation/angry/1177.jpg'] 



for image_path in image_path:
    predicted_emotion = predict_emotion(image_path, model)
    if predicted_emotion:
        print(f"Predicted emotion for the image {image_path}:    {predicted_emotion}")
    else:
        print("Could not predict emotion for the image.", image_path)



Predicted emotion for the image Dataset/images/validation/fear/21.jpg:    fear
Predicted emotion for the image Dataset/images/validation/happy/331.jpg:    happy
Predicted emotion for the image Dataset/images/validation/sad/822.jpg:    neutral
Predicted emotion for the image Dataset/images/validation/neutral/1045.jpg:    neutral
Predicted emotion for the image Dataset/images/validation/surprise/3874.jpg:    surprise
Predicted emotion for the image Dataset/images/validation/angry/1177.jpg:    angry
